## Prepare Environment

In [1]:
# !pip3 install pymongo
# !pip3 install mongoengine
# !pip3 install Faker
# !pip3 install mongomock

In [2]:
# %load_ext nb_black
# %load_ext autoreload
# %autoreload 2

In [3]:
# Prepare environment for importing from src
import sys
import os

sys.path.insert(0, "..")

## Import Dependencies 

In [4]:
import random
import datetime

from mongoengine import connect, get_connection

from src.data import initialize_db
from src.utils import drop_db

## Connect to Mock DB

In [5]:
from pymongo import MongoClient

client = connect("assignment", host="mongodb://127.0.0.1:27017")

In [6]:
# if not os.environ.get("TEST"):
#     drop_db(client, "assignment")

## Generate Fake Data & Insert Them to DB

In [7]:
# if not os.environ.get("TEST"):
#     initialize_db()

In [8]:
print(client.assignment.list_collection_names())
print(client.assignment.patient.find_one())

['pharmacy', 'product', 'company', 'driver', 'comment', 'contract', 'patient', 'product_item', 'drug', 'doctor', 'prescription', 'user', 'sale']
{'_id': ObjectId('6019958165a03929b15c6aed'), 'national_id': '3389083863', 'first_name': 'Shane', 'last_name': 'Ramirez', 'address': '65423 Rogers Common Apt. 594\nBlairmouth, ME 72858', 'birthdate': datetime.datetime(1978, 3, 3, 0, 0), 'password': 'raymond57', 'doctor_id': ObjectId('6019958165a03929b15c6aec')}


# Functions

In [9]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
prprint = pp.pprint

## Examples

In [31]:
ls = (list(client.assignment.drug.find({"formula": "CH3COOH"})))
# prprint(ls)

In [36]:
ls = list(
    client.assignment.drug.aggregate(
        [{"$group": {"_id": "$formula", "count": {"$sum": 1}}}]
    )
)

# prprint(ls)

In [37]:
client.assignment.patient.aggregate(
    [
        {
            "$lookup": {
                "from": "doctor",
                "localField": "doctor_id",
                "foreignField": "_id",
                "as": "doctor",
            }
        },
        {"$match": {"doctor.first_name": "Robert"}},
        {"$count": "patients"},
    ]
).next()

{'patients': 9}

## Query Assignments

In [15]:
# نام داروخانه هایی که شماره تلفن آنها با 1+ شروع می شود
print("##1##")
a1 = list(
    client.assignment.pharmacy.find(
        filter={"telephone" : {"$regex": "^\+1"}},  # Complete the filter
        projection={"name": 1, "_id": 0},
        # projection={"telephone": 2, "name": 1, "_id": 0}
    )
)
print(a1)


##1##
[{'name': 'Rogers-Cardenas'}, {'name': 'Jones, Fox and Romero'}, {'name': 'Lee, Moore and Ferrell'}, {'name': 'Ellison-Arias'}, {'name': 'Henry Inc'}, {'name': 'Lopez-Buchanan'}, {'name': 'Weber-Zamora'}, {'name': 'Kennedy Inc'}, {'name': 'Ramirez Group'}, {'name': 'Fry-Douglas'}, {'name': 'Smith, Kemp and Hughes'}, {'name': 'Nicholson, Smith and Hernandez'}]


In [16]:
# شماره ملی افرادی که بعد از تاریخ datetime.datetime(2000, 1, 1, 0, 0) متولد شده اند
print("##2##")
date = datetime.datetime(2000, 1, 1, 0, 0)
a2 = list(
    client.assignment.patient.find(
        filter={"birthdate": {'$gt': date}},  # Complete the filter
        projection={"national_id": 1, "_id": 0},       
    )
)

print(a2)

##2##
[{'national_id': '7510799118'}, {'national_id': '8069901627'}, {'national_id': '7059640165'}, {'national_id': '9599534283'}, {'national_id': '5973832060'}, {'national_id': '5899441715'}, {'national_id': '9812387800'}, {'national_id': '4271739627'}, {'national_id': '2091241862'}, {'national_id': '0249724399'}, {'national_id': '0242403381'}, {'national_id': '9074260880'}, {'national_id': '3578243477'}, {'national_id': '0961701595'}, {'national_id': '8114578870'}, {'national_id': '8816502544'}, {'national_id': '1123274897'}, {'national_id': '4815269255'}, {'national_id': '8519537749'}, {'national_id': '5700412918'}, {'national_id': '2546862728'}, {'national_id': '8864078802'}, {'national_id': '0186074290'}, {'national_id': '2365401814'}]


In [40]:
# تعداد نسخه هایی که دارای حداقل 15 دارو هستند
print("##3##")
a3 = client.assignment.prescription.find(
    filter={ "items.15" : {u"$exists": True} }  # Complete the filter
).count()

print(a3)


##3##
20


/tmp/ipykernel_141230/1398732763.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  a3 = client.assignment.prescription.find(


In [67]:
# کد ملی بیمارانی که اسم پزشک آنها "Robert" است
print("##4##")

a4 = list(
    client.assignment.patient.aggregate(
    [{
            "$lookup": {
                "from": "doctor",
                "localField": "doctor_id",
                "foreignField": "_id",
                "as": "doctor",
            }
        },
        {"$match": {"doctor.first_name": "Robert"}}, 
        { "$project": { "national_id":1, "_id":0 } }
    ],
    )
)
print(a4)

##4##
[{'national_id': '3389083863'}, {'national_id': '0405411526'}, {'national_id': '9267389975'}, {'national_id': '0242403381'}, {'national_id': '0961701595'}, {'national_id': '2611231261'}, {'national_id': '3153888245'}, {'national_id': '4991004202'}, {'national_id': '0259364502'}]


In [64]:
# نام داروخانه ای که دارویی به گرانترین قیمت به آن فروخته شده است
print("##5##")
a5 = list(client.assignment.sale.aggregate(
    [{
  
        "$lookup": {
            "from": "pharmacy",
            "localField": "pharmacy_id",
            "foreignField": "_id",
            "as": "pha",
        }
    },
    # {'$sort': {'price': -1}},
    # {'$limit': 1},
    ],
))
# ).next()

print(a5)

##5##
[]


In [20]:
# نام و فرمول پنج دارویی که گران ترین قیمت برای آنها ثبت شده است
print("##6##")
a6 = list(
    client.assignment.sale.aggregate(
        [  # Complete the pipeline
        ]
    )
)
# print(a6)

##6##


In [21]:
# نام تمام داروهایی که در تاریخ datetime.datetime(2020, 9, 23, 0, 0) تجویز شده اند
print("##7##")
a7 = list(
    client.assignment.prescription.aggregate(
        [  # Complete the pipeline
        ]
    )
)
# print(a7)

##7##


In [22]:
# نام تمام کارخانه هایی که داروی با فرمول "C2H6Na4O12" را تولید می کنند
print("##8##")
a8 = list(
    client.assignment.drug.aggregate(
        [  # Complete the pipeline
        ]
    )
)
# print(a8)

##8##


In [23]:
# کاربرانی که در سبد آنها ده BasketItem وجود دارد
print("##9##")
a9 = list(
    client.assignment.user.find(
        filter={},  # Complete the filter
        projection={"email": 1, "_id": 0},
    )
)
# print(a9)

##9##


In [24]:
print("##10##")
a10 = client.assignment.product_item.aggregate(
    [  # Complete the pipeline
    ]
).next()
# print(a10)

##10##


In [25]:
# شماره ملی رانندگانی که پلاک آنها به 25 ختم می شود
print("##11##")
a11 = list(
    client.assignment.driver.find(
        filter={},  # Complete the filter
        projection={"_id": 0, "national_id": 1},
    )
)
# print(a11)

##11##


In [26]:
print("##12##")
a12 = list(
    client.assignment.comment.find(
        filter={},  # Complete the filter
        projection={"_id": 0, "text": 1},
    )
)
# print(a12)

##12##


In [27]:
print("##13##")
a13 = client.assignment.comment.aggregate(
    []  # Complete the pipeline
).next()
# print(a13)

##13##


In [28]:
answers = {
    "a1": a1,
    "a2": a2,
    "a3": a3,
    "a4": a4,
    "a5": a5,
    "a6": a6,
    "a7": a7,
    "a8": a8,
    "a9": a9,
    "a10": a10,
    "a11": a11,
    "a12": a12,
    "a13": a13,
}

In [29]:
import json
with open("answers.json", "r") as json_file:
    target = json.load(json_file)

In [30]:
correct = 0
for i in range(1, 14):
    if answers["a{}".format(i)] == target["a{}".format(i)]:
        print("Query {:2d} Correct!".format(i))
        correct += 1
    else:
        print("Query {:2d} Wrong!".format(i))
print(correct)

Query  1 Correct!
Query  2 Correct!
Query  3 Wrong!
Query  4 Wrong!
Query  5 Wrong!
Query  6 Wrong!
Query  7 Wrong!
Query  8 Wrong!
Query  9 Wrong!
Query 10 Wrong!
Query 11 Wrong!
Query 12 Wrong!
Query 13 Wrong!
2


## Print Result to File  

In [ ]:
# Set your student number
student_number = 97106187
file_path = os.path.join(
    os.getenv("OUTPUT_DIR", "."), "{}.json".format(student_number)
)
with open(file_path, "w") as file:
    corrects = []
    wrongs = []
    for i in range(1, 14):
        if answers["a{}".format(i)] == target["a{}".format(i)]:
            corrects.append(i)
        else:
            wrongs.append(i)
    json.dump({"corrects": corrects, "wrongs": wrongs, "score": len(corrects)}, file)